In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if '../' not in sys.path: 
    sys.path.append('../')

In [3]:
from src.srdegrees import SR_Degrees
from src.space_groups import prepare_gap_env, SpaceGroup_Element, SpaceGroup_gap

In [4]:
prepare_gap_env()

In [5]:
G = SpaceGroup_gap.from_gap_cryst(7, dim=2)
G.snot

[[1 0 0]
 [0 1 0]
 [0 0 1],
 [-1  0  0]
 [ 0 -1  0]
 [ 0  0  1],
 [ -1   0 1/2]
 [  0   1   0]
 [  0   0   1],
 [   1    0 -1/2]
 [   0   -1    0]
 [   0    0    1]]

In [53]:
A = G.point_group_normalizer()[0]

conds, base_vars, add_vars = SR_Degrees(7).construct_congruences(A.inverse(), A)
conds

                 [1 0|0]   [1 0|0]
      [1 0 0]    [0 1|0]   [0 1|0]
a_inv [0 1 0]a = [---+-] = [---+-]
      [0 0 1]    [0 0|1]   [0 0|1]

                    [  -1    0|2*a0]   [-1  0| 0]
      [-1  0  0]    [   0   -1|2*a1]   [ 0 -1| 0]
a_inv [ 0 -1  0]a = [---------+----] = [-----+--]
      [ 0  0  1]    [   0    0|   1]   [ 0  0| 1]

                       [            1             0|            0]   [ -1   0
      [ -1   0 1/2]    [            0            -1|2*a1 + 1/2*x1]   [  0   1
a_inv [  0   1   0]a = [---------------------------+-------------] = [-------
      [  0   0   1]    [            0             0|            1]   [  0   0

|1/2]
|  0]
+---]
|  1]

                          [     -1       0|   2*a0]   [   1    0|-1/2]
      [   1    0 -1/2]    [      0       1|-1/2*x1]   [   0   -1|   0]
a_inv [   0   -1    0]a = [---------------+-------] = [---------+----]
      [   0    0    1]    [      0       0|      1]   [   0    0|   1]



[-n0,
 -m0,
 -2*a0 - n1,
 -2*a1 - m1,
 -n2 - 1/2,
 -2*a1 - m2 - 1/2*x1,
 -2*a0 - n3 + 1/2,
 -m3 + 1/2*x1]

In [112]:
def free_add(exp, variables): 
    res = 0
    for v in variables: 
        res += v * exp.coefficient(v)
    return exp - res

for c in conds: 
    print(free_add(c, base_vars + add_vars))

0
0
0
0
-1/2
0
1/2
0


In [145]:


solve([con == 0 for con in conds], add_vars)

[[n0 == 0,
  m0 == 0,
  n1 == -2*a0,
  m1 == -2*a1,
  n2 == (-1/2),
  m2 == -2*a1 - 1/2*x1,
  n3 == -2*a0 + 1/2,
  m3 == 1/2*x1]]

In [150]:
solve_diophantine(A.det() == var('n'))

NotImplementedError: No solver has been written for inhomogeneous_ternary_quadratic.

In [86]:
for el in conds: 
    print(el.solve_diophantine(solution_dict=True))

{n0: 0}
{m0: 0}
{a0: t_0, n1: -2*t_0}
{a1: t_0, m1: -2*t_0}
[]
{a1: t_0, m2: t_1, x1: -4*t_0 - 2*t_1}
[]
{m3: t_0, x1: 2*t_0}


In [115]:
eq = [
    [QQ(free_add(c, base_vars + add_vars))] for c in conds
]

for e, c in zip(eq, conds): 
    for v in (base_vars + add_vars): 
        e.append(QQ(c.coefficient(v)))

eq

[[0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, -2, 0, 0, 0, -1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -2, 0, 0, 0, -1, 0, 0, 0, 0],
 [-1/2, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
 [0, 0, -1/2, 0, -2, 0, 0, 0, 0, 0, -1, 0, 0],
 [1/2, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, -1, 0],
 [0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]]

In [116]:
P = Polyhedron(eqns=eq)
P

A 4-dimensional polyhedron in QQ^12 defined as the convex hull of 1 vertex and 4 lines (use the .plot() method to plot)

In [117]:
P.Hrepresentation()

(An equation (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2) x + 0 == 0,
 An equation (0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0) x - 1 == 0,
 An equation (0, 1, 0, 4, 0, 0, 0, 0, 0, 2, 0, 0) x + 0 == 0,
 An equation (0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0) x + 1 == 0,
 An equation (0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0) x + 0 == 0,
 An equation (0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0) x + 0 == 0,
 An equation (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) x + 0 == 0,
 An equation (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0) x + 0 == 0)

In [118]:
P.Vrepresentation()

(A line in the direction (0, 0, 1, 0, 0, 0, -2, 0, 0, 0, -2, 0),
 A line in the direction (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 A line in the direction (0, 2, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1),
 A line in the direction (0, 4, 0, -1, 0, 0, 0, 2, 0, 0, 0, 2),
 A vertex at (0, 0, 0, 0, 0, 0, 0, 0, -1/2, 0, 1/2, 0))

In [140]:
milp = MixedIntegerLinearProgram(solver='ppl')
milp.set_objective(None)
x = milp.new_variable(integer=True, nonnegative=True)

subs_dict = {
    str(v) : x[i] for i,v in enumerate(base_vars + add_vars)
}

res = []
for c in conds:
    exp = QQ(free_add(c, base_vars + add_vars))
    for v in base_vars + add_vars: 
        exp += subs_dict[str(v)] * c.coefficient(v)
    milp.add_constraint(exp == 0)

milp

Integer Program (no objective, 12 variables, 8 constraints)

In [141]:
milp.solve()

MIPSolverException: PPL : There is no feasible solution

In [123]:
??milp.new_variable

Docstring:
   Return a new "MIPVariable" instance.

   A new variable "x" is defined by:

      sage: p = MixedIntegerLinearProgram(solver='GLPK')
      sage: x = p.new_variable(nonnegative=True)

   It behaves exactly as a usual dictionary would. It can use any key
   argument you may like, as "x[5]" or "x["b"]", and has methods
   "items()" and "keys()".

   See also:

     * "set_min()", "get_min()" -- set/get the lower bound of a
       variable

     * "set_max()", "get_max()" -- set/get the upper bound of a
       variable

   INPUT:

   * "binary", "integer", "real" -- boolean. Set one of these
     arguments to "True" to ensure that the variable gets the
     corresponding type.

   * "nonnegative" -- boolean (default: "False"); whether the variable
     should be assumed to be nonnegative. Rather useless for the
     binary type

   * "name" -- string; associates a name to the variable. This is only
     useful when exporting the linear program to a file using
     "write_mps"

In [120]:
P.generating_function_of_integral_points()

NotImplementedError: cannot compute the generating function of polyhedra with negative coordinates

In [47]:
sc = SR_Degrees(7).algorithm()
sc

=========================== Group 7 ==================================
Generators of group:

[ [-1  0  0]  [ -1   0 1/2]  [1 0 1]  [1 0 0] ]
[ [ 0 -1  0]  [  0   1   0]  [0 1 0]  [0 1 1] ]
[ [ 0  0  1], [  0   0   1], [0 0 1], [0 0 1] ]

SNoT

[ [1 0 0]  [-1  0  0]  [ -1   0 1/2]  [   1    0 -1/2] ]
[ [0 1 0]  [ 0 -1  0]  [  0   1   0]  [   0   -1    0] ]
[ [0 0 1], [ 0  0  1], [  0   0   1], [   0    0    1] ]


[ [x1  0]  [ 0 x1] ]
[ [ 0 x0], [x0  0] ]


----------------Dilation-------------------------

... testing inverse A (should have integral entities):

A^{-1} = 
[x1  0]
[ 0 x0]

                 [1 0|0]   [1 0|0]
      [1 0 0]    [0 1|0]   [0 1|0]
a_inv [0 1 0]a = [---+-] = [---+-]
      [0 0 1]    [0 0|1]   [0 0|1]

                    [  -1    0|2*a0]   [-1  0| 0]
      [-1  0  0]    [   0   -1|2*a1]   [ 0 -1| 0]
a_inv [ 0 -1  0]a = [---------+----] = [-----+--]
      [ 0  0  1]    [   0    0|   1]   [ 0  0| 1]

                       [           -1             0|2*a0 + 1/2*

{'[1/x1    0]\n[   0 1/x0]': (
[1/x1    0]  [x1  0]                                                                                                                      
[   0 1/x0], [ 0 x0], [[n0 == 0, m0 == 0, n1 == -2*a0, m1 == -2*a1, n2 == -2*a0 - 1/2*x1 + 1/2, m2 == 0, n3 == 1/2*x1 - 1/2, m3 == -2*a1]]
)}

In [23]:
A, conds = list(sc.values())[0]
A.charpoly().roots()

[(1/x1, 1), (1/x0, 1)]

In [43]:
SR_Degrees(7, method='latex').sr_degrees()

\newpage
\subsection{Group 7}
Generators of group:
$$\left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
-1 & 0 & \frac{1}{2} \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right]$$
SNoT
$$\left[\begin{array}{l}
\text{\texttt{[1{ }0{ }0]}}\\
\text{\texttt{[0{ }1{ }0]}}\\
\text{\texttt{[0{ }0{ }1]}}
\end{array}, \begin{array}{l}
\text{\texttt{[{-}1{ }{ }0{ }{ }0]}}\\
\text{\texttt{[{ }0{ }{-}1{ }{ }0]}}\\
\text{\texttt{[{ }0{ }{ }0{ }{ }1]}}
\end{array}, \begin{array}{l}
\text{\texttt{[{ }{-}1{ }{ }{ }0{ }1/2]}}\\
\text{\texttt{[{ }{ }0{ }{ }{ }1{ }{ }{ }0]}}\\
\text{\texttt{[{ }{ }0{ }{ }{ }0{ }{ }{ }1]}}
\end{array}, \begin{array}{l}
\text{\texttt{[{ }{ }{ }1{ }{ }{ }{ }0{ }{-}1/2]}}\\
\text{\texttt{[{ }{ }{ }0{ }{ }{ }{-}1{ }{ }{ }{ }0]}}\\
\text{\texttt{[{ }{ }{ }0

In [18]:
A_inv = G.point_group_normalizer()[0]
A = A_inv.inverse()
srd = SR_Degrees(7)
conds, base_vars, variables = srd.construct_congruences(A, A_inv, include_Avar=True)

                           [1 0|0]
      [1 0 0]    [1 0 0]   [0 1|0]
a_inv [0 1 0]a = [0 1 0] = [---+-]
      [0 0 1]    [0 0 1]   [0 0|1]

                                       [-1  0| 0]
      [-1  0  0]    [  -1    0 2*a0]   [ 0 -1| 0]
a_inv [ 0 -1  0]a = [   0   -1 2*a1] = [-----+--]
      [ 0  0  1]    [   0    0    1]   [ 0  0| 1]

                                                                     [ -1   0
      [ -1   0 1/2]    [            1             0             0]   [  0   1
a_inv [  0   1   0]a = [            0            -1 2*a1 + 1/2*x1] = [-------
      [  0   0   1]    [            0             0             1]   [  0   0

|1/2]
|  0]
+---]
|  1]

                                                      [   1    0|-1/2]
      [   1    0 -1/2]    [     -1       0    2*a0]   [   0   -1|   0]
a_inv [   0   -1    0]a = [      0       1 -1/2*x1] = [---------+----]
      [   0    0    1]    [      0       0       1]   [   0    0|   1]



In [33]:
res = srd.solve_congruences(conds, base_vars, variables)
res


equations: 

[                                                                 
[                                                                 
[ -n0 = 0, -m0 = 0, -2*a0 - n1 = 0, -2*a1 - m1 = 0, -n2 - 1/2 = 0,

              x1                                  x1     ]
 -2*a1 - m2 - -- = 0                        -m3 + -- = 0 ]
              2     , -2*a0 - n3 + 1/2 = 0,       2      ]


answer:

[                                                                 x1 
[                                                    m2 = -2*a1 - -- 
[ n0 = 0, m0 = 0, n1 = -2*a0, m1 = -2*a1, n2 = -1/2,              2 ,

                       x1 ]
                  m3 = -- ]
 n3 = 1/2 - 2*a0,      2  ]

[*] Contradiction: n2 == (-1/2)!


In [11]:
for el in res[0]: 
    if el.left() in variables and el.right().is
el

x0 == r24

In [30]:
t = res[0][4].right()
not t.is_integer() and not t.variables()

True

In [22]:
[conds[0].coefficient(v) for v in variables]

[0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0]

In [8]:
help(IntegerLattice)

Help on function IntegerLattice in module sage.modules.free_module_integer:

IntegerLattice(basis, lll_reduce=True)
    Construct a new integer lattice from ``basis``.

    INPUT:

    - ``basis`` -- can be one of the following:

      - a list of vectors

      - a matrix over the integers

      - an element of an absolute order

    - ``lll_reduce`` -- boolean (default: ``True``); run LLL reduction on the basis
      on construction

    EXAMPLES:

    We construct a lattice from a list of rows::

        sage: from sage.modules.free_module_integer import IntegerLattice
        sage: IntegerLattice([[1,0,3], [0,2,1], [0,2,7]])
        Free module of degree 3 and rank 3 over Integer Ring
        User basis matrix:
        [-2  0  0]
        [ 0  2  1]
        [ 1 -2  2]

    Sage includes a generator for hard lattices from cryptography::

        sage: from sage.modules.free_module_integer import IntegerLattice
        sage: A = sage.crypto.gen_lattice(type='modular', m=10, seed=1337

In [7]:
from sage.modules.free_module_integer import IntegerLattice 

help(matrix(QQ, [[1, 2], [2, 1]]).LLL)

Help on built-in function LLL:

LLL(...) method of sage.matrix.matrix_rational_dense.Matrix_rational_dense instance
    Matrix_rational_dense.LLL(self, *args, **kwargs)
    File: sage/matrix/matrix_rational_dense.pyx (starting at line 2942)

            Return an LLL reduced or approximated LLL reduced lattice for
            ``self`` interpreted as a lattice.

            The arguments ``*args`` and ``**kwargs`` are passed onto
            :meth:`sage.matrix.matrix_integer_dense.Matrix_integer_dense.LLL`,
            see there for more details.

            EXAMPLES::

                sage: A = Matrix(QQ, 3, 3, [1/n for n in range(1, 10)])
                sage: A.LLL()
                [ 1/28 -1/40 -1/18]
                [ 1/28 -1/40  1/18]
                [    0 -3/40     0]
                sage: L, U = A.LLL(transformation=True)
                sage: U * A == L
                True

                sage: A = random_matrix(QQ, 10, 10)
                sage: d = lcm(a.denom() for a in A

In [23]:
srd.solve_congruences_v2(conds, variables)

NotImplementedError: only integer lattices supported

In [6]:
srd = SR_Degrees(7)
srd.algorithm()

=========================== Group 7 ==================================
Generators of group:

[ [-1  0  0]  [ -1   0 1/2]  [1 0 1]  [1 0 0] ]
[ [ 0 -1  0]  [  0   1   0]  [0 1 0]  [0 1 1] ]
[ [ 0  0  1], [  0   0   1], [0 0 1], [0 0 1] ]

SNoT

[ [1 0 0]  [-1  0  0]  [ -1   0 1/2]  [   1    0 -1/2] ]
[ [0 1 0]  [ 0 -1  0]  [  0   1   0]  [   0   -1    0] ]
[ [0 0 1], [ 0  0  1], [  0   0   1], [   0    0    1] ]


[ [x0  0]  [ 0 x1] ]
[ [ 0 x1], [x0  0] ]


----------------Dilation-------------------------

... testing inverse A (should have integral entities):

A^{-1} = 
[x0  0]
[ 0 x1]

                           [1 0|0]
      [1 0 0]    [1 0 0]   [0 1|0]
a_inv [0 1 0]a = [0 1 0] = [---+-]
      [0 0 1]    [0 0 1]   [0 0|1]

                                       [-1  0| 0]
      [-1  0  0]    [  -1    0 2*a0]   [ 0 -1| 0]
a_inv [ 0 -1  0]a = [   0   -1 2*a1] = [-----+--]
      [ 0  0  1]    [   0    0    1]   [ 0  0| 1]

                                                               